In [7]:
import pyautogui as pag
from time import sleep
from selenium import webdriver
import numpy as np
from selenium.webdriver.support import expected_conditions as EC
import copy
import csv
import datetime
from time import sleep

In [4]:
web = webdriver.Chrome("/Users/yasha_ten/Downloads/chromedrv")

In [15]:
from collections import defaultdict
import random
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
SELENIUM_WAIT_TIMEOUT = 10



class SingleMatchParser:
    def __init__(self, url, driver):
        self.url = url
        self.dict = None
        driver.switch_to.window(driver.current_window_handle)
        self.driver = driver
        self.score_sets = None
        self.sets_duration = None
        self.match_duration = None
        self.sets_stat = None
        self.serving_idxs = None
        self.k1 = None
        self.k2 = None
        self.status = "Ended"
        
    def wait_until_visible(*conditions):
        def decorator(function):
            def wrapper(self, *args, **kwargs):
                for cond in conditions:
                    WebDriverWait(self.driver, SELENIUM_WAIT_TIMEOUT).until(
                        EC.presence_of_all_elements_located(cond))
                function(self, *args, **kwargs)
            return wrapper
        return decorator
    
    def wait_until_cond(self, css):
        flag = 0
        while True:
            try:
                self.driver.find_element_by_css_selector(css)
                break
            except Exception:
                if (flag < 20):
                    sleep(0.3)
                    flag += 1
                else:
                    raise Exception("TimeLimit")
                   
    def find_status(self):
        wait_until_cond("div.Cell-decync-0.fUNPnK")
        text = self.driver.find_elements_by_css_selector("div.Cell-decync-0.fUNPnK")[0].text.split()
        if "Walkover" in text:
            self.status = "Walkover"
        if "Retired" in text:
            self.status = "Retired"
        if "Canceled" in text:
            self.status = "Canceled"
    
    def scroll_page(self):
        page_height = self.driver.execute_script("return document.body.scrollHeight")
        pos = 0
        step = 500
        while pos < page_height:
            self.driver.execute_script("window.scrollTo(0, {p})".format(p=pos))
            time.sleep(0.5)
            pos += step
    
    
    def get_player_names(self):
        wait_until_cond("h2.styles__PageTitle-rztuto-1.dgCfTI")
        text = self.driver.find_element_by_css_selector("h2.styles__PageTitle-rztuto-1.dgCfTI").text
        assert text != '' and ' - ' in text ### CHECK
        player1, player2 = text.split(' - ')
        self.player2idx = {player1.lower(): 1, player2.lower(): 2}
        self.player1 = player1
        self.player2 = player2
    
    def get_total_score(self):
        wait_until_cond("div.styles__StyledResult-sc-171us6i-4.hbbalZ")
        text = self.driver.find_element_by_css_selector("div.styles__StyledResult-sc-171us6i-4.hbbalZ").text
        assert text != '' and ' - ' in text ### CHECK
        self.score1, self.score2 = text.split(' - ')
    
    def get_match_timings(self):
        wait_until_cond("div.Cell-decync-0.fUNPnK")
        text = self.driver.find_elements_by_css_selector("div.Cell-decync-0.fUNPnK")[0].text
        assert text != '' and '\n' in text ### CHECK
        ### TO DO: format processing
        if "after" in text.split('\n')[1]:
            self.match_duration = text.split('\n')[1]
        self.date = text.split('\n')[0]
    
    def get_sets_score(self):
        try:
            wait_until_cond("tbody.styles__TBody-sc-1vgi783-3.bjSKpE")
            score = []
            for x in self.driver.find_element_by_css_selector("tbody.styles__TBody-sc-1vgi783-3.bjSKpE").text.split(" "):
                if ((("7" in x.split()[0] and "6" in x.split()[1]) or ("6" in x.split()[0] and "7" in x.split()[1]))
                    and (len(x.split()) > 2)):
                    temp1 = x.split()[0][0]
                    temp2 = x.split()[1][0]
                    temp1 += ("(" + x.split()[0][1] + ")")
                    temp2 += ("(" + x.split()[1][1] + ")")
                    score.append(tuple([temp1, temp2]))
                else:
                    score.append(tuple([x.split()[0], x.split()[1]]))
            self.score_sets = score
        except NoSuchElementException:
            pass
    

    def get_sets_time(self):
        try:
            res = []
            arr = self.driver.find_element_by_css_selector("tfoot.styles__TFoot-sc-1vgi783-4.fwVyrl").text
            assert arr != '' ### CHECK
            arr = arr.split()
            for t in range(1, len(arr), 2):
                hrs = int(arr[t - 1][0])
                m = int(arr[t][:-1])
                time = hrs * 60 + m
                res.append(str(time))
            self.sets_duration = res
        except NoSuchElementException:
            pass
    
    def get_sets_stat(self):
        try:
            block = self.driver.find_elements_by_css_selector("a.Label-sc-19k9vkh-0.bfqsCw")
            if block != []:
                stat = defaultdict(dict)
                for tab in block:
                    tab.click()
                    tab_name = tab.text
                    rows = self.driver.find_elements_by_css_selector("div.Cell-decync-0.StatisticsStyles__StatisticsItemCell-zf4n59-2.dihIaT")
                    for row in rows:
                        p1, metric, p2 = row.text.split('\n')
                        stat[tab_name][metric] = [p1, p2]
                self.sets_stat = stat
        except NoSuchElementException:
            pass
    

    def get_odds(self):
        try:
            text = self.driver.find_element_by_css_selector("div.Cell-decync-0.fUNPnK.u-mV8").text.split()
            assert text != '' and len(text) == 4 ### Check
            self.k1 = float(text[1])
            self.k2 = float(text[3])
        except NoSuchElementException:
            pass
    
    
    def get_match_info(self):
        info = dict()
        rows = self.driver.find_elements_by_css_selector("div.styles__MatchInfoRow-sc-1nav912-1.gkEvHG")
        for row in rows:
            records = row.text.split('\n')
            for rec in records:
                assert ': ' in rec ### CHECK
                k, v = rec.split(': ')
                info[k] = v
        if "Venue" in info:
            del info["Venue"]
        if "Start date" in info:
            del info["Start date"]
        self.match_info = info

    
    def parse_page(self):
        self.find_status()
        if (self.status != "Canceled"):
            self.scroll_page()
            self.get_player_names()
            self.get_match_timings()
            self.get_odds()
            self.get_match_info()
            if (self.status != "Walkover"):
                self.get_sets_time()
                self.get_sets_score()
                self.get_sets_stat()
        
    def as_dict(self):
        self.parse_page()
        if self.status != "Canceled":
            self.dict = {
                'match_url': self.url, 
                'player1': self.player1, 
                'player2': self.player2,
                'status': self.status,
                'match_info': self.match_info,
                'date': self.date,
                'match_duration': self.match_duration,
                'score_sets': self.score_sets,
                'sets_duration': self.sets_duration,
                'sets_stat': self.sets_stat,
                'serving_idxs': self.serving_idxs,
                'k1': self.k1,
                'k2': self.k2,
            }
        return self.dict
    

def wait_until_cond(css):
        flag = 0
        while True:
            try:
                web.find_element_by_css_selector(css)
                break
            except NoSuchElementException:
                if (flag < 20):
                    sleep(0.3)
                    flag += 1
                else:
                    raise Exception("TimeLimit")

In [16]:
match = SingleMatchParser("https://www.sofascore.com/tsitsipas-rublev/fKRsqXY", web).as_dict()

In [17]:
match

{'match_url': 'https://www.sofascore.com/tsitsipas-rublev/fKRsqXY',
 'player1': 'Tsitsipas S.',
 'player2': 'Rublev A.',
 'status': 'Ended',
 'match_info': {'Location': 'London ', 'Surface': 'Hardcourt indoor'},
 'date': '1',
 'match_duration': None,
 'score_sets': [('6', '1')],
 'sets_duration': ['96', '60'],
 'sets_stat': defaultdict(dict,
             {'ALL': {'Aces': ['3', '4'],
               'Double faults': ['0', '1'],
               'First serve': ['61/83 (73%)', '66/95 (69%)'],
               'Second serve': ['22/22 (100%)', '28/29 (97%)'],
               'Points won': ['95', '83'],
               'Break points': ['2/9 (22%)', '1/1 (100%)'],
               'First serve points': ['49/61 (80%)', '46/66 (70%)'],
               'Second serve points': ['12/22 (55%)', '15/29 (52%)'],
               'Max points in a row': ['7', '9'],
               'Receiver points won': ['34', '22']},
              '1ST': {'Aces': ['1', '0'],
               'Double faults': ['0', '0'],
             

In [20]:
## Функция нахождения css ссылок для нажатия на матч
def find_css_urls():
    css_urls = web.find_elements_by_css_selector("a.EventCellstyles__Link-sc-1m83enb-0.dhKVQJ")
    return css_urls

## Преход на страницу матча
def go_to_match():
    wait_until_cond("a.styles__EventLink-d389b-0.koniBB")
    elem = web.find_element_by_css_selector("a.styles__EventLink-d389b-0.koniBB")
    elem.click()

## Поиск лузера    
def set_loser(loser_name, match): 
    wait_until_cond("div.Content-sc-1o55eay-0.EventCellstyles__WinIndicator-ni00fg-4.kCvfzg")
    if loser_name == match["player1"]:
        match["player1_win"] = 0
    else:
        match["player1_win"] = 1
    
## Открытие вкладки справа    
def open_tab(css_urls, parsed):
    flag, css, match_url = 0, "", ""
    for css in css_urls:
        match_url = css.get_attribute("href")
        if match_url not in parsed:
            parsed.append(match_url)
            css.click()
            flag = 1
            break
    return [flag, css, match_url]


def is_single():
    wait_until_cond("a.styles__EventLink-d389b-0.koniBB")
    text = web.find_element_by_css_selector("a.styles__EventLink-d389b-0.koniBB").text
    assert text != '' ### CHECK
    if ("Doubles" in text) or ("Mixed" in text) or ("." not in text):
        return 0
    return 1
        
## Скроллим для прогрузки матчей    
def scroll_page(pos):
    t = 0
    while t < pos + 300:
        web.execute_script("window.scrollTo(0, {p})".format(p=t))
        t += 400
        sleep(0.5)
    sleep(0.5)
    
def find_stage():
    el = web.find_element_by_css_selector("ul.BreadcrumbContent__Content-ciuw58-0.styles__WidgetBreadcrumb-d389b-5.ghJOmO")
    return el.text.split()[-1]

## Поиск раунда
def set_stage(stage, match):
    match["round"] = stage
    
## Добавление ссылок    
def add_hrefs(hrefs):
    elements = web.find_elements_by_css_selector("div.styles__TeamSectionMain-sc-171us6i-1.eIMsEV a")
    player1 = elements[-2].get_attribute("href")
    player2 = elements[-1].get_attribute("href")
    hrefs.add(player1)
    hrefs.add(player2)
        
## Парсим страницу определенной даты
def parse_date(date, matches, hrefs):
    web.switch_to.window(web.current_window_handle)
    pos = 0
    parsed = []
    page_height = web.execute_script("return document.body.scrollHeight") ## Высота страницы
    url = "https://www.sofascore.com/tennis/atp/" + date  ## Страница с матчами
    web.get(url)
    web.execute_script("window.scrollTo(0, {p})".format(p=0)) ## Переход в начало страницы
    while True:
        sleep(0.7)
        css_urls = find_css_urls()
        sleep(0.5)
        flag, css, match_url = open_tab(css_urls, parsed)
        if flag == 1:
            sleep(0.7) ## Sleep без которого не работают следующие 2 строки
            if is_single():
                add_hrefs(hrefs)
                stage = find_stage()
                loser_name = css.find_element_by_css_selector("div.Content-sc-1o55eay-0.EventCellstyles__WinIndicator-ni00fg-4.kCvfzg").text ## Поиск лузера            
                go_to_match()
                sleep(0.7) ## Sleep без которого не успевает прогрузиться страница для корректного скроллинга
                match = SingleMatchParser(match_url, web).as_dict() ## Парсинг страницы матча -> match
                web.get(url) ## Возврат на основную страницу парсинга
                sleep(0.7)
                if match != None:
                    set_loser(loser_name, match)
                    set_stage(stage, match)
                    matches.append(match)
            pos += 50
            scroll_page(pos)  
            flag = 0
        else:
            break  ## Выход из парсинга страницы, когда не осталось новых ссылок в css_urls
    print("Parsed matches on date: {d}".format(d=date), len(parsed))

In [22]:
hrefs=set()
matches=[]
parse_date("2020-11-11", matches, hrefs)

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=87.0.4280.88)


In [ ]:
matches=[]
hrefs=set()

In [23]:
## Парсинг по датам, вернет matches - массив матчей, hrefs - сет ссылок игроков
def parse(startdate, enddate, matches, hrefs):
    while startdate != enddate:
        parse_date(str(startdate), matches, hrefs) 
        startdate += datetime.timedelta(1)

[{'match_url': 'https://www.sofascore.com/millman-simon/LUfspmn',
  'player1': 'Millman J.',
  'player2': 'Simon G.',
  'status': 'Ended',
  'match_info': {'Location': 'Sofia ', 'Surface': 'Hardcourt indoor'},
  'date': '11 Nov 2020, 13:00',
  'match_duration': 'Ended after 3h 15m',
  'score_sets': [('7', '5')],
  'sets_duration': ['68', '80'],
  'sets_stat': defaultdict(dict,
              {'ALL': {'Aces': ['3', '5'],
                'First serve': ['71/99 (72%)', '62/118 (53%)'],
                'Second serve': ['28/28 (100%)', '56/56 (100%)'],
                'Points won': ['114', '103'],
                'Break points': ['6/16 (38%)', '3/5 (60%)'],
                'First serve points': ['43/71 (61%)', '40/62 (65%)'],
                'Second serve points': ['19/28 (68%)', '26/56 (46%)'],
                'Max points in a row': ['6', '8'],
                'Receiver points won': ['52', '37']},
               '1ST': {'Aces': ['0', '2'],
                'Double faults': ['0', '0'],
      